In [1]:
# !pip install ../../OSAR-keras/.

# Comparison Example for OSAR, DQN and C51 agents

In [2]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents import agents
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

from OSAR import OSARNetwork, Runner, DQNTrialAgent

print(tf.version.VERSION)

2.7.0


2021-11-22 19:24:53.840994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-22 19:24:53.848693: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-22 19:24:53.849160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-22 19:24:53.850344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [4]:
env_names = [
    'KukaBulletEnv-v0',
]

In [5]:
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 1000  # @param {type:"integer"}

batch_size = 1  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 20  # @param {type:"integer"}
memory_len = 10 # @param {type: "integer"}
n_turns = 3 # @param {type: "integer"}
num_atoms = 51  # @param {type:"integer"}
q_value = 10  # @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}

num_eval_episodes = 20  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

boltzmann_temperature = 10e-5 # @param {type:"float"}
epsilon_greedy = None # @param {type:"float"}
conv_type = '2d' # @param {type:"str"}

In [6]:
fc_layer_params = (64, 64)
from tf_agents.networks import categorical_q_network, q_network

osar_specs = {
    'batch_size': batch_size,
    'memory_len': memory_len,
    'n_turns': n_turns,
    'fc_layer_params': fc_layer_params,
    'num_atoms': num_atoms,
    'conv_type': conv_type,
    'learning_rate': learning_rate,
    'q_value': q_value,
    'n_step_update': n_step_update,
    'boltzmann_temperature': boltzmann_temperature,
    'epsilon_greedy': epsilon_greedy,
    'debug_summaries': True,
    'summarize_grads_and_vars': True,
}

def osar_generator(
    observation_spec,
    action_spec,
    batch_size,
    memory_len,
    n_turns,
    fc_layer_params,
    num_atoms,
    conv_type,
    learning_rate,
    time_step_spec,
    n_step_update,
    train_step_counter,
    q_value=q_value,
    boltzmann_temperature=None,
    epsilon_greedy=0.1,
    debug_summaries=True,
    summarize_grads_and_vars=True,
    **kwargs,
    ):
    q_net = OSARNetwork(
        observation_spec,
        action_spec,
        batch_size,
        memory_len,
        n_turns,
        fc_layer_params=fc_layer_params,
        conv_type=conv_type,
        )
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    agent = DQNTrialAgent(
        time_step_spec,
        action_spec,
        network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        boltzmann_temperature=boltzmann_temperature,
        epsilon_greedy=epsilon_greedy,
        debug_summaries=debug_summaries,
        train_step_counter = tf.Variable(0, dtype=tf.int64),
        summarize_grads_and_vars=summarize_grads_and_vars,
    )
    
    agent.initialize()
    
    return agent

    

In [7]:
experiment_common_specs = {
    'agent_specs': osar_specs,
    'agent_generator': osar_generator,
    'num_iterations': num_iterations,
    'initial_collect_steps': initial_collect_steps,
    'collect_steps_per_iteration': collect_steps_per_iteration,
    'replay_buffer_max_length': replay_buffer_max_length,
    'num_eval_episodes': num_eval_episodes,
    'eval_interval': eval_interval,
    'n_step_update': n_step_update,
}

In [8]:
configs = []
for name in env_names:
    config = experiment_common_specs.copy()
    config['env_name'] = name
    configs.append(config)

In [9]:
%%time
logpath = ''
model_name = 'kuka_osar'
runner = Runner(model_name=model_name, logpath=logpath, list_configs=configs)
runner.run(progress=False, experiment_progress=True)

pybullet build time: Oct 11 2021 21:00:24


current_dir=/home/constantine/miniconda3/envs/agents/lib/python3.9/site-packages/pybullet_envs/bullet


/home/constantine/miniconda3/envs/agents/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


BoundedArraySpec(shape=(3,), dtype=dtype('int32'), name='action', minimum=0, maximum=99)


ValueError: Network only supports action_specs with shape in [(), (1,)])
  In call to configurable 'OSARNetwork' (<class 'OSAR.network.OSARNetwork'>)
  In call to configurable 'Experiment' (<class 'OSAR.runner.Experiment'>)

In [10]:
fc_layer_params = (64, 64)
from tf_agents.networks import categorical_q_network, q_network

c51_specs = {
    'batch_size': batch_size,
    'memory_len': memory_len,
    'n_turns': n_turns,
    'fc_layer_params': fc_layer_params,
    'num_atoms': num_atoms,
    'conv_type': conv_type,
    'learning_rate': learning_rate,
    'q_value': q_value,
    'n_step_update': n_step_update,
    'boltzmann_temperature': boltzmann_temperature,
    'epsilon_greedy': epsilon_greedy,
    'debug_summaries': True,
    'summarize_grads_and_vars': True,
}
def c51_generator(
    observation_spec,
    action_spec,
    batch_size,
    memory_len,
    n_turns,
    fc_layer_params,
    num_atoms,
    learning_rate,
    time_step_spec,
    n_step_update,
    conv_type,
    train_step_counter,
    q_value=q_value,
    boltzmann_temperature=None,
    epsilon_greedy=0.1,
    debug_summaries=True,
    summarize_grads_and_vars=True,
    ):

    q_net = categorical_q_network.CategoricalQNetwork(
    observation_spec,
    action_spec,
    num_atoms=num_atoms,
    fc_layer_params=fc_layer_params)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    n_step_update = 1
    
    agent = agents.categorical_dqn.categorical_dqn_agent.CategoricalDqnAgent(
        time_step_spec,
        action_spec,
        categorical_q_network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        train_step_counter = tf.Variable(0, dtype=tf.int64),
        min_q_value = -q_value,
        max_q_value = q_value,
        n_step_update = n_step_update,
        boltzmann_temperature=boltzmann_temperature,
        epsilon_greedy=epsilon_greedy,
        debug_summaries=debug_summaries,
        summarize_grads_and_vars=summarize_grads_and_vars,
    )
    agent.initialize()
    
    return agent

experiment_c51_specs = {
    'agent_specs': c51_specs,
    'agent_generator': c51_generator,
    'num_iterations': num_iterations,
    'initial_collect_steps': initial_collect_steps,
    'collect_steps_per_iteration': collect_steps_per_iteration,
    'replay_buffer_max_length': replay_buffer_max_length,
    'num_eval_episodes': num_eval_episodes,
    'eval_interval': eval_interval,
    'n_step_update': n_step_update,
}
configs = []
for name in env_names:
    config = experiment_c51_specs.copy()
    config['env_name'] = name
    configs.append(config)

In [11]:
%%time
logpath = ''
model_name = 'kuka_c51'
runner = Runner(model_name=model_name, logpath=logpath, list_configs=configs)
runner.run(progress=False, experiment_progress=True)

BoundedArraySpec(shape=(3,), dtype=dtype('int32'), name='action', minimum=0, maximum=99)


ValueError: Only scalar actions are supported now, but action spec is: BoundedTensorSpec(shape=(3,), dtype=tf.int32, name='action', minimum=array(0, dtype=int32), maximum=array(99, dtype=int32))
  In call to configurable 'DqnAgent' (<class 'tf_agents.agents.dqn.dqn_agent.DqnAgent'>)
  In call to configurable 'CategoricalDqnAgent' (<class 'tf_agents.agents.categorical_dqn.categorical_dqn_agent.CategoricalDqnAgent'>)
  In call to configurable 'Experiment' (<class 'OSAR.runner.Experiment'>)

In [12]:
fc_layer_params = (64, 64)
from tf_agents.networks import categorical_q_network, q_network

dqn_specs = {
    'batch_size': batch_size,
    'memory_len': memory_len,
    'n_turns': n_turns,
    'fc_layer_params': fc_layer_params,
    'num_atoms': num_atoms,
    'conv_type': conv_type,
    'learning_rate': learning_rate,
    'q_value': q_value,
    'n_step_update': n_step_update,
    'boltzmann_temperature': boltzmann_temperature,
    'epsilon_greedy': epsilon_greedy,
    'debug_summaries': True,
    'summarize_grads_and_vars': True,
}

def dqn_generator(
    observation_spec,
    action_spec,
    batch_size,
    memory_len,
    n_turns,
    fc_layer_params,
    num_atoms,
    conv_type,
    learning_rate,
    time_step_spec,
    n_step_update,
    train_step_counter,
    q_value=10,
    boltzmann_temperature=None,
    epsilon_greedy=0.1,
    debug_summaries=True,
    summarize_grads_and_vars=True,
    ):

    q_net = q_network.QNetwork(
        observation_spec,
        action_spec,
        fc_layer_params=fc_layer_params)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    n_step_update = 1
    
    agent = agents.dqn.dqn_agent.DqnAgent(
        time_step_spec,
        action_spec,
        q_network=q_net,
        optimizer=optimizer,
        td_errors_loss_fn=common.element_wise_squared_loss,
        train_step_counter = tf.Variable(0, dtype=tf.int64),
        n_step_update = n_step_update,
        boltzmann_temperature=boltzmann_temperature,
        epsilon_greedy=epsilon_greedy,
        debug_summaries=debug_summaries,
        summarize_grads_and_vars=summarize_grads_and_vars,
    )
    agent.initialize()
    
    return agent

experiment_dqn_specs = {
    'agent_specs': dqn_specs,
    'agent_generator': dqn_generator,
    'num_iterations': num_iterations,
    'initial_collect_steps': initial_collect_steps,
    'collect_steps_per_iteration': collect_steps_per_iteration,
    'replay_buffer_max_length': replay_buffer_max_length,
    'num_eval_episodes': num_eval_episodes,
    'eval_interval': eval_interval,
    'n_step_update': n_step_update,
}
configs = []
for name in env_names:
    config = experiment_dqn_specs.copy()
    config['env_name'] = name
    configs.append(config)

In [13]:
%%time
logpath = ''
model_name = 'kuka_dqn'
runner = Runner(model_name=model_name, logpath=logpath, list_configs=configs)
runner.run(progress=False, experiment_progress=True)

BoundedArraySpec(shape=(3,), dtype=dtype('int32'), name='action', minimum=0, maximum=99)


ValueError: Network only supports action_specs with shape in [(), (1,)])
  In call to configurable 'QNetwork' (<class 'tf_agents.networks.q_network.QNetwork'>)
  In call to configurable 'Experiment' (<class 'OSAR.runner.Experiment'>)

In [14]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard

In [15]:
# %tensorboard --logdir logs

In [16]:
# !kill 1369

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
dqn_reward = pd.read_csv('logs/kuka_dqn/datasets/KukaCamBulletEnv-v0-steps.csv').loc[:, 'Uclipped Return Train'].values.cumsum()
osar_reward = pd.read_csv('logs/kuka_osar/datasetsKukaCamBulletEnv-v0-steps.csv').loc[:, 'Uclipped Return Train'].values.cumsum()
c51_reward = pd.read_csv('logs/kuka_c51/datasets/KukaCamBulletEnv-v0-steps.csv').loc[:, 'Uclipped Return Train'].values.cumsum()

plt.figure(figsize=(7, 8))
plt.xlabel = 'Episode'
plt.ylabel = 'Unclipped Summary Reward'
plt.plot(dqn_reward, 'r', label='DQN')
plt.plot(c51_reward, 'b', label='C51')
plt.plot(osar_reward, 'k', label='OSAR')
plt.grid()
plt.legend()
plt.savefig('logs/cum_reward_kuka.png')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'logs/kuka_dqn/datasets/KukaCamBulletEnv-v0-steps.csv'